# 1. Imports

In [11]:
from src.configuration import Configuration
from src.fetch_data import DataFetcher
from src.dim import Dims
from src.facts import Facts
import pandas as pd
import time
import datetime
from pyspark.sql import functions as f 
from pyspark.sql import  SparkSession, Window
from aggregate_data import DataAggregator



pd.set_option('display.max_columns', None)



# 2. Variables

In [12]:
config = Configuration('src/config.cfg')
fetcher = DataFetcher(config)
DIMS = Dims(config, fetcher)
FACTS = Facts(config, fetcher)
DATAAGGREGATOR = DataAggregator(config, fetcher)
spark = SparkSession.builder.appName("MojaSesja").master("local").getOrCreate()



25/10/21 19:59:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/10/21 19:59:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# 1. Assigning points throught the season to the driver.

In [3]:
dim_session = DIMS.dim_sessions(race="Race")
fact_session_result = FACTS.fact_session_results()

In [4]:
fact_session_result['position'] = fact_session_result['position'].fillna(value=21)
dim_driver_team = DIMS.dim_driver_team()

In [5]:
dim_session

,session_key,location,date_start,date_end,session_name,country_code,country_name,year,is_current_season,key
1,7953,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023
3,7779,Jeddah,2023-03-19,2023-03-19,Race,KSA,Saudi Arabia,2023,0,Jeddah2023
5,7787,Melbourne,2023-04-02,2023-04-02,Race,AUS,Australia,2023,0,Melbourne2023
9,9070,Baku,2023-04-30,2023-04-30,Race,AZE,Azerbaijan,2023,0,Baku2023
11,9078,Miami,2023-05-07,2023-05-07,Race,USA,United States,2023,0,Miami2023
...,...,...,...,...,...,...,...,...,...,...
149,9928,Budapest,2025-08-03,2025-08-03,Race,HUN,Hungary,2025,1,Budapest2025
151,9920,Zandvoort,2025-08-31,2025-08-31,Race,NED,Netherlands,2025,1,Zandvoort2025
153,9912,Monza,2025-09-07,2025-09-07,Race,ITA,Italy,2025,1,Monza2025
155,9904,Baku,2025-09-21,2025-09-21,Race,AZE,Azerbaijan,2025,1,Baku2025


In [6]:
joined_results  = fact_session_result.merge(
    dim_session,
    on="session_key",
    how="inner"
).merge(
    dim_driver_team,
    on = ['session_key','driver_number'],
    how = "inner"
)



In [7]:
joined_results

,position,driver_number,number_of_laps,dnf,dns,dsq,duration,gap_to_leader,meeting_key,session_key,points,location,date_start,date_end,session_name,country_code,country_name,year,is_current_season,key,team_name
0,1.0,1,57.0,False,False,False,5636.736,0,1141,7953,25.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Red Bull Racing
1,2.0,11,57.0,False,False,False,5648.723,11.987,1141,7953,18.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Red Bull Racing
2,3.0,14,57.0,False,False,False,5675.373,38.637,1141,7953,15.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Aston Martin
3,4.0,55,57.0,False,False,False,5684.788,48.052,1141,7953,12.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Ferrari
4,5.0,44,57.0,False,False,False,5687.713,50.977,1141,7953,10.0,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023,Mercedes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,21.0,12,23.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Mercedes
1273,21.0,6,17.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Racing Bulls
1274,21.0,5,3.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Kick Sauber
1275,21.0,30,0.0,True,False,False,None,None,1277,9947,0.0,Silverstone,2025-07-06,2025-07-06,Race,GBR,United Kingdom,2025,1,Silverstone2025,Racing Bulls


In [8]:
joined_results = joined_results[['position','driver_number','session_key','points','year','is_current_season','key','date_end','team_name']]
spark_df = spark.createDataFrame(joined_results)
spark_df.show()

+--------+-------------+-----------+------+----+-----------------+----------+----------+---------------+
|position|driver_number|session_key|points|year|is_current_season|       key|  date_end|      team_name|
+--------+-------------+-----------+------+----+-----------------+----------+----------+---------------+
|     1.0|            1|       7953|  25.0|2023|                0|Sakhir2023|2023-03-05|Red Bull Racing|
|     2.0|           11|       7953|  18.0|2023|                0|Sakhir2023|2023-03-05|Red Bull Racing|
|     3.0|           14|       7953|  15.0|2023|                0|Sakhir2023|2023-03-05|   Aston Martin|
|     4.0|           55|       7953|  12.0|2023|                0|Sakhir2023|2023-03-05|        Ferrari|
|     5.0|           44|       7953|  10.0|2023|                0|Sakhir2023|2023-03-05|       Mercedes|
|     6.0|           18|       7953|   8.0|2023|                0|Sakhir2023|2023-03-05|   Aston Martin|
|     7.0|           63|       7953|   6.0|2023|       

In [9]:
window = Window.partitionBy("driver_number","year").orderBy("date_end").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

spark_df = spark_df.withColumn("cumulative_points", f.sum("points").over(window))

In [10]:
spark_df.show(50)

+--------+-------------+-----------+------+----+-----------------+--------------------+----------+---------------+-----------------+
|position|driver_number|session_key|points|year|is_current_season|                 key|  date_end|      team_name|cumulative_points|
+--------+-------------+-----------+------+----+-----------------+--------------------+----------+---------------+-----------------+
|     1.0|            1|       7953|  25.0|2023|                0|          Sakhir2023|2023-03-05|Red Bull Racing|             25.0|
|     2.0|            1|       7779|   0.0|2023|                0|          Jeddah2023|2023-03-19|Red Bull Racing|             25.0|
|     1.0|            1|       7787|  25.0|2023|                0|       Melbourne2023|2023-04-02|Red Bull Racing|             50.0|
|     2.0|            1|       9070|  18.0|2023|                0|            Baku2023|2023-04-30|Red Bull Racing|             68.0|
|     1.0|            1|       9078|  26.0|2023|                0|   

In [11]:
window2 = Window.partitionBy("year","team_name").orderBy("date_end")

test = spark_df.withColumn(
    "team_points",
    f.sum("points").over(window2)
)

In [12]:
test.show()

+--------+-------------+-----------+------+----+-----------------+---------------+----------+----------+-----------------+-----------+
|position|driver_number|session_key|points|year|is_current_season|            key|  date_end| team_name|cumulative_points|team_points|
+--------+-------------+-----------+------+----+-----------------+---------------+----------+----------+-----------------+-----------+
|    16.0|           24|       7953|   0.0|2023|                0|     Sakhir2023|2023-03-05|Alfa Romeo|              0.0|        4.0|
|     8.0|           77|       7953|   4.0|2023|                0|     Sakhir2023|2023-03-05|Alfa Romeo|              4.0|        4.0|
|    13.0|           24|       7779|   0.0|2023|                0|     Jeddah2023|2023-03-19|Alfa Romeo|              0.0|        4.0|
|    18.0|           77|       7779|   0.0|2023|                0|     Jeddah2023|2023-03-19|Alfa Romeo|              4.0|        4.0|
|     9.0|           24|       7787|   2.0|2023|       

In [13]:
DATAAGGREGATOR.get_racer_team_points(racer_team="team")

,session_key,key,driver_number,team_name,team_points_gained
0,7953,Sakhir2023,77,Alfa Romeo,4.0
1,7953,Sakhir2023,24,Alfa Romeo,4.0
2,7779,Jeddah2023,24,Alfa Romeo,4.0
3,7779,Jeddah2023,77,Alfa Romeo,4.0
4,7787,Melbourne2023,24,Alfa Romeo,6.0
...,...,...,...,...,...
1272,9912,Monza2025,55,Williams,83.0
1273,9904,Baku2025,55,Williams,98.0
1274,9904,Baku2025,23,Williams,98.0
1275,9896,Marina Bay2025,55,Williams,99.0


In [14]:
DATAAGGREGATOR.get_last_races_result(n_races=5, race_type="Race", measure="position")

,driver_number,position,key,last_race_pos_1,last_race_pos_2,last_race_pos_3,last_race_pos_4,last_race_pos_5
5,1,1.0,Monaco2023,1.0,2.0,1.0,2.0,1.0
6,1,1.0,Barcelona2023,2.0,1.0,2.0,1.0,1.0
7,1,1.0,Montréal2023,1.0,2.0,1.0,1.0,1.0
8,1,1.0,Spielberg2023,2.0,1.0,1.0,1.0,1.0
9,1,1.0,Silverstone2023,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
1272,87,21.0,Budapest2025,17.0,11.0,11.0,11.0,11.0
1273,87,6.0,Zandvoort2025,11.0,11.0,11.0,11.0,21.0
1274,87,12.0,Monza2025,11.0,11.0,11.0,21.0,6.0
1275,87,12.0,Baku2025,11.0,11.0,21.0,6.0,12.0


# 2. Calculate Difference in points between drivers

In [15]:
test = DATAAGGREGATOR.get_racer_team_points(racer_team="driver").merge(
    DIMS.dim_driver_team(),
    on=["driver_number","session_key"]
)

time.sleep(10)


merged = test.merge(
    test,
    on=["key","team_name"],
    suffixes=["","_other"]
)

merged = merged[merged["driver_number"]!= merged["driver_number_other"]]

In [16]:
merged

,session_key,key,driver_number,points_gained,team_name,session_key_other,driver_number_other,points_gained_other
1,7953,Sakhir2023,1,25.0,Red Bull Racing,7953,11,18.0
3,7779,Jeddah2023,1,25.0,Red Bull Racing,7779,11,43.0
5,7787,Melbourne2023,1,50.0,Red Bull Racing,7787,11,54.0
7,9070,Baku2023,1,68.0,Red Bull Racing,9070,11,79.0
9,9078,Miami2023,1,94.0,Red Bull Racing,9078,11,97.0
...,...,...,...,...,...,...,...,...
2541,9928,Budapest2025,87,6.0,Haas F1 Team,9928,31,23.0
2543,9920,Zandvoort2025,87,14.0,Haas F1 Team,9920,31,24.0
2545,9912,Monza2025,87,14.0,Haas F1 Team,9912,31,24.0
2547,9904,Baku2025,87,14.0,Haas F1 Team,9904,31,24.0


In [17]:
merged

,session_key,key,driver_number,points_gained,team_name,session_key_other,driver_number_other,points_gained_other
1,7953,Sakhir2023,1,25.0,Red Bull Racing,7953,11,18.0
3,7779,Jeddah2023,1,25.0,Red Bull Racing,7779,11,43.0
5,7787,Melbourne2023,1,50.0,Red Bull Racing,7787,11,54.0
7,9070,Baku2023,1,68.0,Red Bull Racing,9070,11,79.0
9,9078,Miami2023,1,94.0,Red Bull Racing,9078,11,97.0
...,...,...,...,...,...,...,...,...
2541,9928,Budapest2025,87,6.0,Haas F1 Team,9928,31,23.0
2543,9920,Zandvoort2025,87,14.0,Haas F1 Team,9920,31,24.0
2545,9912,Monza2025,87,14.0,Haas F1 Team,9912,31,24.0
2547,9904,Baku2025,87,14.0,Haas F1 Team,9904,31,24.0


# 3. Calculate gap to leader

In [18]:
racer_points = DATAAGGREGATOR.get_racer_team_points()

# Calculate max points gained per session

max_points_per_session = racer_points.groupby(
    "key"
).agg(
    {'points_gained':'max'}
).reset_index()

In [19]:
racer_points.head()

,session_key,key,driver_number,points_gained
0,7953,Sakhir2023,1,25.0
1,7779,Jeddah2023,1,25.0
2,7787,Melbourne2023,1,50.0
3,9070,Baku2023,1,68.0
4,9078,Miami2023,1,94.0


In [20]:
merged_results = racer_points.merge(
    max_points_per_session,
    on="key",
    how="inner",
    suffixes=["_racer","_total"]
)

In [21]:
merged_results["gap_to_leader"] = merged_results["points_gained_total"] - merged_results["points_gained_racer"]

In [13]:
DIMS.dim_sessions()

,session_key,location,date_start,date_end,session_name,country_code,country_name,year,is_current_season,key
0,7768,Sakhir,2023-03-04,2023-03-04,Qualifying,BRN,Bahrain,2023,0,Sakhir2023
1,7953,Sakhir,2023-03-05,2023-03-05,Race,BRN,Bahrain,2023,0,Sakhir2023
2,7775,Jeddah,2023-03-18,2023-03-18,Qualifying,KSA,Saudi Arabia,2023,0,Jeddah2023
3,7779,Jeddah,2023-03-19,2023-03-19,Race,KSA,Saudi Arabia,2023,0,Jeddah2023
4,7783,Melbourne,2023-04-01,2023-04-01,Qualifying,AUS,Australia,2023,0,Melbourne2023
...,...,...,...,...,...,...,...,...,...,...
157,9896,Marina Bay,2025-10-05,2025-10-05,Race,SGP,Singapore,2025,1,Marina Bay2025
158,9879,Austin,2025-10-17,2025-10-17,Sprint Qualifying,USA,United States,2025,1,Austin2025
159,9883,Austin,2025-10-18,2025-10-18,Sprint,USA,United States,2025,1,Austin2025
160,9884,Austin,2025-10-18,2025-10-18,Qualifying,USA,United States,2025,1,Austin2025
